# Autores colombianos por provincias

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores8.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 2.02 s, sys: 33.1 ms, total: 2.05 s
Wall time: 2.05 s


In [3]:
autores_df.shape

(61857, 10)

In [4]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber,Berlin,Madrid,"(52.52000659999999, 13.404954)","(40.4167754, -3.7037902)"
1,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom,Helen Cooper,London,Madrid,"(51.5073509, -0.1277583)","(40.4167754, -3.7037902)"
2,Codrilo,2009,Spain,Roberto Aliaga Sánchez,Spain,Roberto Aliaga,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
3,"Félix, : el coleccionista de miedos",2009,Spain,Fina Casalderrey,Spain,Fina Casalderrey,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
4,En la laguna más profunda,2011,Colombia,Oscar Collazos,Colombia,Oscar Collazos,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"


## Autores colombianos

In [5]:
autores_colombianos_df = autores_df[autores_df['PAIS_AUTOR'] == 'Colombia']
autores_colombianos_df = autores_colombianos_df.copy()

In [6]:
autores_colombianos_df.shape

(3450, 10)

In [7]:
autores_colombianos_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
4,En la laguna más profunda,2011,Colombia,Oscar Collazos,Colombia,Oscar Collazos,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
177,Zoro,1997,Colombia,Jairo Aníbal Niño,Colombia,Jairo Aníbal Niño,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
232,Mitos y leyendas indígenas de Colombia,2013,Colombia,Javier Ocampo López,Colombia,Javier Ocampo López,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
286,Puro pueblo,2002,Colombia,Jairo Aníbal Niño,Colombia,Jairo Aníbal Niño,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
304,"Octubre, 1942-1952",1953,Colombia,Carmelina Soto Valencia,Colombia,Carmelina Soto Valencia,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"


## Ciudades y departamentos

In [8]:
def get_ciudad_departamento_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            {{
                {{
                    ?author wdt:P106 wd:Q36180 .
                }}
                UNION
                {{ 
                    ?author wdt:P106 ?prof .
                    ?prof wdt:P279 wd:Q36180 .
                }}
            }}
            ?author wdt:P19 ?pob .
            {{
                ?pob wdt:P31 wd:Q2555896 .
            }}
            UNION
            {{
                ?pob wdt:P31 wd:Q5119 .
                ?pob wdt:P17 wd:Q739 .
            }}
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob wdt:P131 ?dep .
                ?dep wdt:P31 wd:Q215655 .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [9]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_ciudad_departamento_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX yago: <http://dbpedia.org/class/yago/>

        SELECT DISTINCT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            {{
                {{
                    ?author rdf:type dbo:Writer .
                }}
                UNION
                {{
                    ?author rdf:type ?prof .
                    ?prof rdfs:subClassOf dbo:Writer .
                }}
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type yago:WikicatMunicipalitiesOfColombia .
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob dbo:isPartOf ?dep .
                ?dep rdf:type yago:WikicatDepartmentsOfColombia .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [10]:
error_autores = set()

def get_ciudad_departamento_autor_data(db, name, lang):
    pob = 'UNKNOWN'
    dep = 'UNKNOWN'
    try:
        if db == 'wdt':
            data = get_ciudad_departamento_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_ciudad_departamento_autor_data_dbp(name, lang)
        
        results = data['results']['bindings']
        if results:
            if len(results) > 1:
                print('MáS DE UNA CIUDAD Y/O DEPARTAMENTO PARA', name, ':', results)
            r = results[0]
            if 'pobLabel' in r:
                pob = r['pobLabel']['value']
            else:
                pob = 'UNKNOWN'
            if 'depLabel' in r:
                dep = r['depLabel']['value']
            else:
                dep = 'UNKNOWN'
    except Exception as e:
        print('ERROR EN SPARQL PARA', name)
        error_autores.add(name)
    return (pob, dep)

In [11]:
def get_ciudad_departamento_autor(name):
    result = ('DESCONOCIDO', 'DESCONOCIDO')
    for lang in ['es', 'en']:
        for db in ['wdt', 'dbp']:
            r = get_ciudad_departamento_autor_data(db, name, lang)
            if r[0] != 'UNKNOWN':
                if r[1] == 'UNKNOWN':
                    result = (r[0], 'DESCONOCIDO')
                else: #r[1] != 'UNKNOWN'
                    result = r
                    return result
    return result

In [12]:
%%time

autores_ciudades_departamentos_dict = {}

for x in autores_colombianos_df['NOMBRE_LOCALIZADO'].unique():
    autores_ciudades_departamentos_dict[x] = get_ciudad_departamento_autor(x)

CPU times: user 7.76 s, sys: 196 ms, total: 7.96 s
Wall time: 1min 50s


In [13]:
%%time

for x in error_autores:
    autores_ciudades_departamentos_dict[x] = get_ciudad_departamento_autor(x)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.29 µs


In [14]:
%%time

def process_ciudad(x):
    return autores_ciudades_departamentos_dict[x['NOMBRE_LOCALIZADO']][0]

autores_colombianos_df['CIUDAD_AUTOR'] = autores_colombianos_df.apply(process_ciudad, axis=1)

CPU times: user 97.5 ms, sys: 0 ns, total: 97.5 ms
Wall time: 96.4 ms


In [15]:
%%time

def process_departamento(x):
    return autores_ciudades_departamentos_dict[x['NOMBRE_LOCALIZADO']][1]

autores_colombianos_df['DEPARTAMENTO_AUTOR'] = autores_colombianos_df.apply(process_departamento, axis=1)

CPU times: user 75.9 ms, sys: 0 ns, total: 75.9 ms
Wall time: 74.3 ms


# Saving results

In [16]:
autores_colombianos_df.to_csv('autores_colombianos_1.csv', index=False)